### Embedded deep feature selection

In [ ]:
import tensorflow as tf
import numpy as np
import random as rn
seed = 1
rand = np.random.RandomState(seed)
np.random.seed(1)
rn.seed(1)
# Setting the graph-level random seed.
tf.random.set_seed(1)
import os
os.environ['PYTHONHASHSEED']=str(1)
import keras
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Concatenate, Layer, Multiply, Reshape, ThresholdedReLU, Dot, \
Activation,LocallyConnected1D,Input, ActivityRegularization, Dropout,  Dense, Lambda
from tensorflow.keras import regularizers, losses
from tensorflow.keras.utils import to_categorical, model_to_dot, plot_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import clone_model
import pandas as pd
from scipy.io import loadmat
from scipy.io.arff import loadarff 
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
import statistics
import pprint
from datetime import datetime


In [ ]:
from generate import *
!cat /proc/cpuinfo
!cat /proc/meminfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
!pip install skrebate
from skrebate import ReliefF

### Classes

In [ ]:
from tensorflow.keras.constraints import MinMaxNorm
class PairLayer(tf.keras.layers.Layer):

    def __init__(self, activation='sigmoid', cancelout_loss=True, lambda_1=0.002, lambda_2=0.001, lambda_3 = 0.0, init=None):
        super(PairLayer, self).__init__()
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2
        self.lambda_3 = lambda_3
        self.cancelout_loss = cancelout_loss
        
        if activation == 'sigmoid': self.activation = tf.sigmoid
        elif activation == 'softmax': self.activation = tf.nn.softmax
        else: self.activation = None
        if init == 'glorout' : self.init = tf.keras.initializers.GlorotNormal()
        else: self.init =  tf.keras.initializers.Constant(1)


    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1],),
            initializer=self.init,
            trainable=True,
            constraint=MinMaxNorm(min_value = 0, max_value = 1, rate = 1, axis = 0))
        
    def call(self, inputs):
        if self.cancelout_loss:
            shap = self.w.shape[0]
            self.add_loss( self.lambda_1 * tf.norm(self.w, ord=1)  + self.lambda_2 * tf.norm(self.w, ord=2))
            if self.lambda_3 != 0.0:
                self.add_loss(-self.lambda_3 * tf.math.reduce_variance(self.w*(1/shap)))

        if self.activation == None: return inputs * self.w
        else: return inputs * self.activation(self.w)
    
    def get_config(self):
        return {"activation": self.activation} 

In [ ]:
class PairwiseFS(Model):
    def __init__(self, inputs, base_model, l1, l2=0.001, l3=0.0, activationCancel='sigmoid', initializer=None):
        self.l1 = l1
        self.inputDim = inputs.shape[1]
        dense = [] 
        inputLayer = Input(shape=(self.inputDim,))

        ##Construct pairwise
        cancel = PairLayer(activation=activationCancel, lambda_1=l1, lambda_2=l2, lambda_3=l3, init=initializer) (inputLayer)
        second = clone_model(base_model) 
        super().__init__(inputs=inputLayer, outputs=second (cancel))


    def get_support(self):
        weights = self.get_weights()[0]
        return weights

In [ ]:
class NFSLayer(tf.keras.layers.Layer):
    def __init__(self, activation='sigmoid'):
        super(NFSLayer, self).__init__()
        
    def call(self, inputs):
        return inputs[1] * inputs[0]


In [ ]:
class NeuralFS(Model):

    def __init__(self, inputs, base_model,l2, dense, l1=0.0, threshold=0.05):
        self.l2 = l2
        self.l1 = l1
        self.inputDim = inputs.shape[1]
        self.inputTensor = inputs
        self.denseHidden = dense
        self.threshold = threshold
        inputLayer = Input(shape=(self.inputDim,))
        dense1 = Dense(self.denseHidden, activation="relu", kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l1_l2(l1=self.l1, l2=self.l2))(inputLayer)
        dense2 = Dense(self.inputDim, activation=self.tr, kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l1_l2(l1=self.l1, l2=self.l2))(dense1) #TL
        pairwise =NFSLayer()([inputLayer, dense2])
        second = clone_model(base_model) 
        ret = second (pairwise)
        super().__init__(inputs=inputLayer, outputs=second (pairwise))

    @tf.function
    def tr(self,x):
        theta = K.cast(self.threshold, K.floatx())
        a = K.cast(K.less(x,-theta) , K.floatx())
        b = K.cast(K.greater(x,theta), K.floatx())
        d = 1 -((1-a)*(1-b)) #A or B
        return x * d


    def get_support(self):
        inputLayer = Input(shape=(self.inputDim,))
        dense1 = Dense(self.denseHidden, activation="relu", \
            weights=[self.get_weights()[0], self.get_weights()[1]])(inputLayer)
        dense2 = Dense(self.inputDim, activation=self.tr, \
                    weights=[self.get_weights()[2], self.get_weights()[3]])(dense1) #TL
        support = Model(inputs=inputLayer, outputs=dense2)
        support.compile(optimizer='adam',
                    loss=losses.CategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])
        prediction = support.predict(self.inputTensor)
        supportList = np.mean(prediction, axis=0)
        return supportList

### Generate dataset

In [ ]:
#dataset, y, df, yb = generateSyntheticDataset(5000); dfstr="synth" #0-4 relevant 5-9 irrelevant 10-13 redondunt
#dataset, y, df = generateCovidDataset(); dfstr="covid"
#dataset, y, df = generateCardioDataset(); dfstr="cardio"
#dataset, y, df = generateDivorceDataset(); dfstr="divorce"
#dataset, y, df = generateChildhoodTumor(); dfstr="childTum"
dataset, y, df = generateGDataset("breast-data.csv","breast-labels.csv" ); dfstr="breast"
#dataset, y, df = generateGDataset("breast-data.csv","breast-labels.csv",breast_feature ); dfstr="breast"
#dataset, y, df = generateGDataset("gravier-data.csv","gravier-labels.csv" ); dfstr="gravier"
#dataset, y, df = generateGDataset("arcene-data.csv","arcene-labels.csv" ); dfstr="arcene"
#dataset, y, df = generateGDataset("lsvt-data.csv","lsvt-labels.csv" ); dfstr="lsvt"
#dataset, y, df = generateGDataset("sonar-data.csv","sonar-labels.csv" ); dfstr="sonar"
#dataset, y, df = generateGDataset("gastro-data.csv","gastro-labels.csv" ); dfstr="gastro"
#dataset, y, df = generateGDataset("alon-data.csv","alon-labels.csv" ); dfstr="alon"
#dataset, y, df = generateGDataset("synth-data.csv","synth-labels.csv" ); dfstr="synth"

X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=0.33, shuffle=False, random_state=1)
#print(X_train)

1      1
2      1
3      1
4      1
5      1
      ..
564    1
565    1
566    1
567    1
568    0
Name: x, Length: 568, dtype: int64
     X17.99  X10.38  X122.8   X1001  ...  X0.7119  X0.2654  X0.4601  X0.1189
1     20.57   17.77  132.90  1326.0  ...   0.2416   0.1860   0.2750  0.08902
2     19.69   21.25  130.00  1203.0  ...   0.4504   0.2430   0.3613  0.08758
3     11.42   20.38   77.58   386.1  ...   0.6869   0.2575   0.6638  0.17300
4     20.29   14.34  135.10  1297.0  ...   0.4000   0.1625   0.2364  0.07678
5     12.45   15.70   82.57   477.1  ...   0.5355   0.1741   0.3985  0.12440
..      ...     ...     ...     ...  ...      ...      ...      ...      ...
564   21.56   22.39  142.00  1479.0  ...   0.4107   0.2216   0.2060  0.07115
565   20.13   28.25  131.20  1261.0  ...   0.3215   0.1628   0.2572  0.06637
566   16.60   28.08  108.30   858.1  ...   0.3403   0.1418   0.2218  0.07820
567   20.60   29.33  140.10  1265.0  ...   0.9387   0.2650   0.4087  0.12400
568    7.76   24.54

### Utils

In [ ]:
def activation_potential_analysis(X, weights):
    weights_first = weights[0]
    bias = weights[1]
    support = []
    p_ij = np.zeros((X.shape[1], len(weights_first[0])))
    for dim in range(len(weights_first)):
        for hidden in range(len(weights_first[0])):
            for trains in range(X.shape[0]):
                p_ij[dim][hidden] += abs(weights_first[dim][hidden]*X[trains][dim])
            p_ij[dim][hidden] = p_ij[dim][hidden]/X.shape[0] 
            
    sum_j = np.sum(p_ij, axis=1)
    return sum_j

In [ ]:
def crossValScore(modelKey,k,X_train,y_train, mapModel, param, epc, bs, verb):
    n_splits = k
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    loss = []
    acc = []
    val_loss = []
    val_acc = []
    final_acc = []
    for train_index, val_index in kf.split(X_train):
        X_train_scal = MinMaxScaler().fit_transform(X_train)
        model = mapModel[modelKey](X_train_scal[train_index],y_train[train_index],param)
        model.compile(optimizer='adam',
                      loss=losses.CategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])
        hist = model.fit(X_train_scal[train_index], y_train[train_index],
                         epochs=epc,
                         batch_size=bs,
                         validation_data = (X_train_scal[val_index], y_train[val_index]),
                         verbose=verb)

        loss.append(hist.history['loss'])
        acc.append(hist.history['accuracy'])
        val_loss.append([hist.history['val_loss']])
        val_acc.append(hist.history['val_accuracy'])
        final_acc.append(hist.history['accuracy'][-1])
        score = np.mean(final_acc)
    return score, acc, loss, final_acc
#print(val_acc)

In [ ]:
def filterImportance(X_train,y_train):
    yb = y_train[:,1]
    fsMut = SelectKBest(score_func=mutual_info_classif, k='all')
    fsMut.fit(X_train, yb)
    fsR = ReliefF(n_neighbors=1, n_features_to_select=X_train.shape[1])
    X_train = fsR.fit_transform(X_train, yb)
    fsMSc = fsMut.scores_*(1/(np.amax(fsMut.scores_)))
    fsRSc = fsR.feature_importances_*(1/(np.amax(fsR.feature_importances_)))
    fsUnion = fsMSc + fsRSc
    return fsUnion

In [ ]:
def showFeatures(key,w):
    print(w)
    if X_train.shape[1]<200:
        fig = plt.figure(figsize=(8, 8))
        fig.suptitle(key, fontsize=16)
        ax = fig.add_axes([0,0,1,1])
        if X_train.shape[1]>200:
            print("too much dimension to display")
        elif X_train.shape[1]>20:
            ax.barh(df.columns,[abs(i) for i in w])
        else:
            ax.bar(df.columns,[abs(i) for i in w])
        plt.show()

In [ ]:
def computeSupport(w, n):
    sortedFeat = sorted(w)
    thres = sortedFeat[-n]
    support = []
    print(w)
    for i in w:
        if i < thres or i <= 0.0:
            support.append(False)
        else:
            support.append(True)
    pres = support.count(True)
    if pres < n:
        count = 0
        ind = 0
        while count < n-pres:
            if support[ind] == False:
                count += 1
                support[ind] = True
            ind += 1
    return support



In [ ]:
def computeSupportPercent(w, p):
    sortedFeat = sorted(w)
    sortedFeat.reverse()
    sumFeat = np.sum(w)
    thresPerc = sumFeat*(p/100)
    support = []
    thres = 0
    for i in range(len(sortedFeat)):
        thres += sortedFeat[i]
        if thres+(1*(10**(-5))) >= round(thresPerc,7):
            return computeSupport(w,i+1)
    return False

In [ ]:
def writeOut(name, params, result, dfstr):
    with open(name, "a") as file:
        file.write("###")
        file.write("\n")
        file.write(dfstr)
        file.write("\n")
        now = datetime.now()
        timestamp = datetime.timestamp(now)
        file.write(str(now))
        file.write("\n")
        file.write(str(params))
        file.write("\n")
        file.write(str(result))
        file.write("\n")


### Models

Choose a suitable baseline architecture manually

In [ ]:
def createBaseline(X_train,y_train,params):
    baseline = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(params["dense1_baseline"],activation='relu', kernel_regularizer=regularizers.l1_l2(l1=params["l1_baseline"], l2=params["l2_baseline"]), kernel_initializer='glorot_normal'),
        Dense(params["dense2_baseline"],activation='relu', kernel_regularizer=regularizers.l1_l2(l1=params["l1_baseline"], l2=params["l2_baseline"]),  kernel_initializer='glorot_normal'),
        Dense(params["dense3_baseline"],activation='relu', kernel_regularizer=regularizers.l1_l2(l1=params["l1_baseline"], l2=params["l2_baseline"]),  kernel_initializer='glorot_normal'),
        Dense(y.shape[1], activation='softmax')
    ])
    return baseline




In [ ]:
def createRegularized(X_train,y_train,params):
    regularized = Sequential([
    Dense(X_train.shape[1],activation='relu', kernel_regularizer=regularizers.l1(params["l1_regul"]), input_shape=(X_train.shape[1],)),
    clone_model(createBaseline(X_train,y_train,params))
    ])
    return regularized

In [ ]:
def createDropout(X_train,y_train,params):
    dropout = Sequential([
    Dropout(.5),
    clone_model(createBaseline(X_train,y_train,params))
    ])
    return dropout

In [ ]:
def createCancelOut(X_train,y_train,params):
    cancelOut =  PairwiseFS(inputs=X_train, base_model=createBaseline(X_train,y_train,params), l1=params["l1_cancelOut"], l2=params["l2_cancelOut"], l3=params["l3_cancelOut"], initializer=params["initialiser_cancelOut"])
    return cancelOut


In [ ]:
def createPairwise(X_train,y_train,params):
    pairwise = PairwiseFS(inputs=X_train, base_model=createBaseline(X_train,y_train,params), l1=params["l1_pairwise"], l2=params["l2_pairwise"], l3=params["l3_pairwise"], activationCancel=None, initializer=params["initialiser_pairwise"])
    return pairwise

In [ ]:
def createNeuralFS(X_train,y_train, params):
    neuralFS = NeuralFS(inputs=X_train, base_model=createBaseline(X_train,y_train,params), l2=params["l2_neuralfs"], dense=X_train.shape[1], l1=params["l1_neuralfs"],threshold=params["threshold_neuralfs"])
    return neuralFS


In [ ]:
#Run this cell if you change any model
mapModel = {'neuralFS': createNeuralFS, "dropout": createDropout, "pairwise" : createPairwise,
           "cancelOut" : createCancelOut, "baseline": createBaseline, "regularized": createRegularized }

### Performance

In [ ]:
def testModels(X_train, y_train, modelList,params,epc,percent=False,nFeat=None, showFeat=False, cols=False):
    retMap = {}
    baseline = createBaseline(X_train,y_train,params)
    baseline.compile(optimizer='adam',
    loss=losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])
    baseline.fit(X_train,y_train,epochs=epc, verbose=0)
    test_loss, test_acc = baseline.evaluate(X_test,  y_test, verbose=1)
    retMap["shape"] = X_train.shape
    retMap["test_acc_baseline"] = test_acc
    y_pred1 = baseline.predict(X_test)
    y_pred = np.argmax(y_pred1, axis=1)
    y_true = np.argmax(y_test, axis=1)
    bcr = balanced_accuracy_score(y_true, y_pred)
    retMap["test_bcr_baseline"] = bcr
    for key in modelList:
        print("$$__"+key)
        if key != "filter":
            #print(mapModel)
            model = mapModel[key](X_train,y_train,params)
            model.compile(optimizer='adam',
              loss=losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
            model.fit(X_train,y_train,epochs=epc, verbose=0)
            test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)
            y_pred1 = model.predict(X_test)
            y_pred = np.argmax(y_pred1, axis=1)
            y_true = np.argmax(y_test, axis=1)

            bcr = balanced_accuracy_score(y_true, y_pred)
            if key in ["baseline", "regularized", "dropout"]:
                w = activation_potential_analysis(X_train, model.get_weights())
                w = [abs(number) for number in w]
            else:
                w = model.get_support()
                w = [abs(number) for number in w]
            if showFeat:
                showFeatures(key,w)
            retMap["test_acc_full_"+key] = test_acc
            retMap["test_bcr_full_"+key] = bcr
            
        else:
            w = filterImportance(X_train, y_train)
            if showFeat:
                showFeatures(key,w)
        if nFeat != None:
            if percent == True:
                support = computeSupportPercent(w,nFeat)
            else:
                support = computeSupport(w,nFeat)
            print(support)
            print("\n")
            X_train_red = X_train[:,support]
            X_test_red = X_test[:,support]
            retMap["shape_reduced_"+key] = X_train_red.shape
            if cols == True:
                retMap["columns_reduced_"+key] = df.columns[support].values.tolist()
            baseline = createBaseline(X_train_red,y_train,params)
            baseline.compile(optimizer='adam',
                loss=losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
            baseline.fit(X_train_red,y_train,epochs=epc, verbose=0)
            test_loss, test_acc = baseline.evaluate(X_test_red,  y_test, verbose=0)
            retMap["test_acc_baseline_reduced_from_"+key] = test_acc
            y_pred1 = baseline.predict(X_test_red)
            y_pred = np.argmax(y_pred1, axis=1)
            y_true = np.argmax(y_test, axis=1)
            bcr = balanced_accuracy_score(y_true, y_pred)
            retMap["test_bcr_baseline_reduced_from_"+key] = bcr
    return retMap




Find best baseline architecture

In [ ]:
if dfstr == 'arcene':
 params={'dense1_baseline': 10,
 'dense2_baseline': 10,
 'dense3_baseline': 10,
 'initialiser_cancelOut': 'glorout',
 'initialiser_pairwise': 'glorout',
 'k': 3,
 'l1_baseline': 0.001,
 'l1_cancelOut': 0.05,
 'l1_neuralfs': 0.0,
 'l1_pairwise': 0.05,
 'l1_regul': 0.01,
 'l2_baseline': 0.001,
 'l2_cancelOut': 0.0,
 'l2_neuralfs': 0.05,
 'l2_pairwise': 0.0,
 'l3_cancelOut': 0.3,
 'l3_pairwise': 0.0,
 'threshold_neuralfs': 0.005}


elif dfstr == 'synth':

  params = {'dense1_baseline': 100,
 'dense2_baseline': 10,
 'dense3_baseline': 100,
 'initialiser_cancelOut': 'glorout',
 'initialiser_pairwise': 'const',
 'k': 3,
 'l1_baseline': 0.001,
 'l1_cancelOut': 0.05,
 'l1_neuralfs': 0.0,
 'l1_pairwise': 0.05,
 'l1_regul': 0.01,
 'l2_baseline': 0.001,
 'l2_cancelOut': 0.05,
 'l2_neuralfs': 0.05,
 'l2_pairwise': 0.05,
 'l3_cancelOut': 0.3,
 'l3_pairwise': 0.0,
 'threshold_neuralfs': 0.05}

elif dfstr == 'gastro':
  params = {'dense1_baseline': 50,
 'dense2_baseline': 100,
 'dense3_baseline': 50,
 'initialiser_cancelOut': 'const',
 'initialiser_pairwise': 'glorout',
 'k': 3,
 'l1_baseline': 0.001,
 'l1_cancelOut': 0.05,
 'l1_neuralfs': 0.0,
 'l1_pairwise': 0.05,
 'l1_regul': 0.01,
 'l2_baseline': 0.01,
 'l2_cancelOut': 0.05,
 'l2_neuralfs': 0.05,
 'l2_pairwise': 0.0,
 'l3_cancelOut': 0.3,
 'l3_pairwise': 0.0,
 'threshold_neuralfs': 0.01}

elif dfstr == 'breast':
  params={'dense1_baseline': 10,
 'dense2_baseline': 50,
 'dense3_baseline': 100,
 'initialiser_cancelOut': 'const',
 'initialiser_pairwise': 'const',
 'k': 3,
 'l1_baseline': 0.001,
 'l1_cancelOut': 0.5,
 'l1_neuralfs': 0.0,
 'l1_pairwise': 0.05,
 'l1_regul': 0.01,
 'l2_baseline': 0.001,
 'l2_cancelOut': 0.4,
 'l2_neuralfs': 0.5,
 'l2_pairwise': 0.25,
 'l3_cancelOut': 0.6,
 'l3_pairwise': 0.1,
 'threshold_neuralfs': 0.005}
else:
  print("OK")
  params = {  "l1_baseline" : 0.001, "l2_baseline" : 0.001, "dense1_baseline" : 100, "dense2_baseline" : 100, "dense3_baseline" : 100,
              "l1_regul" : 0.03, "l1_cancelOut" : 0.6, "l2_cancelOut" : 0.2, "l3_cancelOut": 0, "initialiser_cancelOut" : 'const',
              "l1_pairwise" : 0.2, "l2_pairwise": 0.0, "l3_pairwise": 0.0, "initialiser_pairwise" : 'glorout', 
              "l1_neuralfs" : 0.00, "l2_neuralfs" : 0.05, "threshold_neuralfs" : 0.05
          }

In [ ]:
pprint.pprint(params)

{'dense1_baseline': 10,
 'dense2_baseline': 50,
 'dense3_baseline': 100,
 'initialiser_cancelOut': 'const',
 'initialiser_pairwise': 'const',
 'k': 3,
 'l1_baseline': 0.001,
 'l1_cancelOut': 0.5,
 'l1_neuralfs': 0.0,
 'l1_pairwise': 0.05,
 'l1_regul': 0.01,
 'l2_baseline': 0.001,
 'l2_cancelOut': 0.4,
 'l2_neuralfs': 0.5,
 'l2_pairwise': 0.25,
 'l3_cancelOut': 0.6,
 'l3_pairwise': 0.1,
 'threshold_neuralfs': 0.005}


### Test 

In [ ]:
pprint.pprint(params)
X_train = MinMaxScaler().fit_transform(X_train)
X_test = MinMaxScaler().fit_transform(X_test)

{'dense1_baseline': 10,
 'dense2_baseline': 50,
 'dense3_baseline': 100,
 'initialiser_cancelOut': 'const',
 'initialiser_pairwise': 'const',
 'k': 3,
 'l1_baseline': 0.001,
 'l1_cancelOut': 0.5,
 'l1_neuralfs': 0.0,
 'l1_pairwise': 0.05,
 'l1_regul': 0.01,
 'l2_baseline': 0.001,
 'l2_cancelOut': 0.4,
 'l2_neuralfs': 0.5,
 'l2_pairwise': 0.25,
 'l3_cancelOut': 0.6,
 'l3_pairwise': 0.1,
 'threshold_neuralfs': 0.005}


In [ ]:
modelList = ['filter','neuralFS','pairwise','cancelOut','regularized','baseline']

result = testModels(X_train, y_train, modelList, params, epc=100, percent=True, nFeat=100, showFeat=False, cols=False)
writeOut("resultFS.txt",params,result,dfstr)


6/6 [==============================] - 0s 2ms/step - loss: 0.1796 - accuracy: 0.9734
$$__filter
[1.32229918 0.90154021 1.4111268  1.31982461 0.49991409 0.65581488
 1.31204402 1.62174508 0.41302612 0.16382061 0.92618526 0.29101552
 0.82894569 1.03347874 0.14855066 0.22885385 0.44124318 0.54286909
 0.23720242 0.19245491 1.98151867 1.29099891 1.95045852 1.91659282
 0.61954095 0.82472499 1.14347457 1.6890153  0.44810302 0.25326948]
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


$$__neuralFS
6/6 [==============================] - 0s 2ms/step - loss: 0.1922 - accuracy: 0.9681
[0.1739624, 0.22651772, 0.07844103, 0.16142355, 0.09911292, 0.044895522, 0.15223017, 0.25742263, 0.07018297, 0.12585561, 0.44929203, 0.032408398, 0.31488034, 0.44068906, 0.018446943, 0.12482932, 0.00723981, 0.08471636, 0.16179079, 0.45810586, 0.24560906, 0.23033246, 0.23807885, 0.406353

In [ ]:
pprint.pprint(result)

{'shape': (380, 30),
 'shape_reduced_baseline': (380, 30),
 'shape_reduced_cancelOut': (380, 29),
 'shape_reduced_filter': (380, 30),
 'shape_reduced_neuralFS': (380, 30),
 'shape_reduced_pairwise': (380, 29),
 'shape_reduced_regularized': (380, 30),
 'test_acc_baseline': 0.9734042286872864,
 'test_acc_baseline_reduced_from_baseline': 0.9734042286872864,
 'test_acc_baseline_reduced_from_cancelOut': 0.9680851101875305,
 'test_acc_baseline_reduced_from_filter': 0.9468085169792175,
 'test_acc_baseline_reduced_from_neuralFS': 0.9627659320831299,
 'test_acc_baseline_reduced_from_pairwise': 0.9680851101875305,
 'test_acc_baseline_reduced_from_regularized': 0.9680851101875305,
 'test_acc_full_baseline': 0.9680851101875305,
 'test_acc_full_cancelOut': 0.9680851101875305,
 'test_acc_full_neuralFS': 0.9680851101875305,
 'test_acc_full_pairwise': 0.9680851101875305,
 'test_acc_full_regularized': 0.9627659320831299,
 'test_bcr_baseline': 0.9827586206896552,
 'test_bcr_baseline_reduced_from_baselin

75% of ranking

In [ ]:
modelList = ['filter','neuralFS','pairwise','cancelOut','regularized','baseline']
print(modelList)
result = testModels(X_train, y_train, modelList, params, epc=100, percent=True, nFeat=75, showFeat=False, cols=False)
pprint.pprint(params)
print("___________________________________")
pprint.pprint(result)
writeOut("resultFS.txt",params,result,dfstr)


['filter', 'neuralFS', 'pairwise', 'cancelOut', 'regularized', 'baseline']
6/6 [==============================] - 0s 3ms/step - loss: 0.1677 - accuracy: 0.9787
$$__filter
[1.33436232 0.90598077 1.40925364 1.32641968 0.48371821 0.65543336
 1.31636468 1.63093259 0.41978698 0.16382061 0.92822161 0.29400524
 0.82541129 1.03623355 0.15192444 0.22858421 0.44033248 0.54565021
 0.23720242 0.18702811 1.99126956 1.29009651 1.95045852 1.92361423
 0.62851909 0.82834218 1.14752505 1.69722446 0.4468897  0.25539919]
[True, True, True, True, False, False, True, True, False, False, True, False, False, True, False, False, False, False, False, False, True, True, True, True, False, True, True, True, False, False]


$$__neuralFS
6/6 [==============================] - 0s 2ms/step - loss: 0.1774 - accuracy: 0.9681
[0.12648952, 0.23166709, 0.0, 0.19979712, 0.06651034, 0.09931911, 0.21397679, 0.25508842, 0.21359311, 0.24745895, 0.46254033, 0.05321328, 0.36351982, 0.5219251, 0.124729455, 0.0, 0.3862574, 0.02352

50% of ranking

In [ ]:
modelList = ['filter','neuralFS','pairwise','cancelOut','regularized','baseline']
print(modelList)
result = testModels(X_train, y_train, modelList, params, epc=100, percent=True, nFeat=50, showFeat=False, cols=False)
pprint.pprint(params)
print("___________________________________")
pprint.pprint(result)
writeOut("resultFS.txt",params,result,dfstr)


['filter', 'neuralFS', 'pairwise', 'cancelOut', 'regularized', 'baseline']
6/6 [==============================] - 0s 2ms/step - loss: 0.1891 - accuracy: 0.9628
$$__filter
[1.32982949 0.90873281 1.40924977 1.31935306 0.47914166 0.65825823
 1.31469162 1.6216802  0.42410512 0.16382061 0.92559591 0.29393508
 0.82822836 1.03471786 0.14716116 0.22959901 0.44222444 0.54458718
 0.23720242 0.18429258 1.98602541 1.28448768 1.95045852 1.91834717
 0.61642194 0.82495927 1.14387837 1.69097386 0.43874128 0.25366422]
[True, False, True, True, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, True, False, False, False, True, False, False]


$$__neuralFS
6/6 [==============================] - 0s 2ms/step - loss: 0.1838 - accuracy: 0.9787
[0.14644384, 0.20909038, 0.0, 0.1543651, 0.11865159, 0.036383323, 0.17286977, 0.2633359, 0.06072307, 0.14532346, 0.37909448, 0.16327971, 0.3635798, 0.4788195, 0.030126577, 0.35177684, 0.09617

Divide the number of feature by two

In [ ]:
modelList = ['filter','neuralFS','pairwise','cancelOut','regularized','baseline']
print(modelList)
result = testModels(X_train, y_train, modelList, params, epc=100, percent=False, nFeat=int((X_train.shape[1])/2), showFeat=False, cols=False)
pprint.pprint(params)
print("___________________________________")
pprint.pprint(result)
writeOut("resultFS.txt",params,result,dfstr)


['filter', 'neuralFS', 'pairwise', 'cancelOut', 'regularized', 'baseline']
6/6 [==============================] - 0s 2ms/step - loss: 0.1789 - accuracy: 0.9840
$$__filter
[1.3308454  0.90375222 1.4134567  1.3273586  0.48837764 0.6632999
 1.31810963 1.63245668 0.41747641 0.16382061 0.92878944 0.29334911
 0.82587049 1.04178948 0.1475705  0.23020061 0.43972067 0.55463867
 0.23720242 0.18763679 1.99086956 1.28568102 1.95045852 1.92337456
 0.6330565  0.82653717 1.14716402 1.69387916 0.4420258  0.25069791]
[True, True, True, True, False, False, True, True, False, False, True, False, False, True, False, False, False, False, False, False, True, True, True, True, False, True, True, True, False, False]


$$__neuralFS
6/6 [==============================] - 0s 2ms/step - loss: 0.1962 - accuracy: 0.9628
[0.15110691, 0.33149546, 0.071325056, 0.21363167, 0.15772104, 0.052302536, 0.19328111, 0.28934395, 0.036869545, 0.08726354, 0.41748038, 0.011070989, 0.3807064, 0.5304593, 0.24729869, 0.31018394, 0.2

### Stability

In [ ]:
def crossStab(key,k,X_train,y_train, mapModel, param, percent, nFeat, epc, bs, verb):
    n_splits = k
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    sups = []
    for train_index, val_index in kf.split(X_train):
        model = mapModel[key](X_train,y_train,params)
        model.compile(optimizer='adam',
          loss=losses.CategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])
        model.fit(X_train,y_train,epochs=epc, verbose=0)
        test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)
        if key in ["baseline", "regularized", "dropout"]:
            w = activation_potential_analysis(X_train, model.get_weights())
            w = [abs(number) for number in w]
        else:
            w = model.get_support()
            w = [abs(number) for number in w]
        if percent == True:
            support = computeSupportPercent(w,nFeat)
        else:
            support = computeSupport(w,nFeat)
        sups.append(support)
    dim = len(sups[0])
    if percent == True:
        index = jaccardIndex(sups,k,dim)
        print("Mean jacc index ",key,": ",index)

    else:
        index = kunchevaIndex(sups,k,dim, nFeat)
        print("Kuncheva index ", key,": ",index)
    return index

In [ ]:
def jaccardIndex(sups, k, dim):
    jacc = 0
    count = 0
    print("h",k)
    print(sups)
    for i in range(k-1):
        for j in range(i+1,k):
            if i != j:
                count += 1
                card = 0
                a = 0
                for l in range(dim):
                    if sups[i][l] == True or sups[j][l] == True:
                        a += 1
                    if sups[i][l] == True and sups[j][l] == True:
                        card += 1
                jacc += (card/(a))
    jacc_index = jacc/count
    return jacc_index

In [ ]:
def kunchevaIndex(sups,k, dim, nFeat):
    interKun = 0
    for i in range(k-1):
        for j in range(i+1,k):
            card = 0
            for l in range(dim):
                if sups[i][l] == True and sups[j][l] == True:
                    card += 1
            interKun += (card - ((nFeat**2)/dim)) / (nFeat - ((nFeat**2)/dim)) 
    kun_index =  2*interKun/(k*(k-1))
    return kun_index

### 100% of ranking

In [ ]:
train = True
modelList = ['baseline','neuralFS','pairwise','cancelOut','regularized']
result = {}
result["shape"] = X_train.shape
if train:
    for key in modelList:
        if key != "filter":
            ind = crossStab(key,4,X_train,y_train,mapModel, params, percent=True, nFeat=100, epc=100, bs=32, verb=0)
            result["stab_"+key] = ind
writeOut("resultFS.txt",params,result,dfstr)

6/6 [==============================] - 0s 2ms/step - loss: 0.2067 - accuracy: 0.9521
[0.34161165987547015, 0.3705245792708202, 0.5003216756278741, 0.1562406490317215, 0.40385067388147455, 0.20438441450616956, 0.42377840992914495, 0.3984786901477446, 0.3670722239386232, 0.43865318272344045, 0.1126674305270172, 0.13138850914676445, 0.1548301338993843, 0.14327935299275038, 0.14182053809023493, 0.20555748972578083, 0.08759440344175679, 0.24007146625854373, 0.122380034189419, 0.08378362658702147, 0.22049948594604687, 0.7470189847056514, 0.34454955620282557, 0.29973671548480907, 0.35098439141955107, 0.18109454048372264, 0.2828034880275433, 0.7555973487959843, 0.27149366409895015, 0.18143187089654705]
6/6 [==============================] - 0s 2ms/step - loss: 0.2214 - accuracy: 0.9574
[0.307814211865885, 0.3865987891706551, 0.21668745376385132, 0.1684197246706857, 0.5499214739365746, 0.29974637082155886, 0.3670132931148949, 0.3666010935870494, 0.3767891325055569, 0.42588382654064233, 0.212107

50% of ranking

In [ ]:
train = True
modelList = ['baseline','neuralFS','pairwise','cancelOut','regularized']
result = {}
result["shape"] = X_train.shape
if train:
    for key in modelList:
        if key != "filter":
            ind = crossStab(key,4,X_train,y_train,mapModel, params, percent=True, nFeat=50, epc=100, bs=32, verb=0)
            result["stab_"+key] = ind
writeOut("resultFS.txt",params,result,dfstr)

6/6 [==============================] - 0s 3ms/step - loss: 0.1777 - accuracy: 0.9628
[0.22533467698587306, 0.28837265134746004, 0.5389418797350294, 0.2305557641694011, 0.5123955914881462, 0.2311751133854506, 0.1431812964539269, 0.45284059453093406, 0.31107600779558575, 0.21768208794590113, 0.14102182848248965, 0.12617385176099968, 0.1723650159816689, 0.07296440881353274, 0.1299944671146848, 0.14247698768359937, 0.04512262245010258, 0.15696839696864978, 0.13596935124240692, 0.0362768027797304, 0.470437635282658, 0.7350088540925735, 0.21850666314525113, 0.3079965381999925, 0.3818208369994397, 0.20298523770880986, 0.23836013043258739, 0.5570467312595503, 0.2042916048625035, 0.07266307940411176]
6/6 [==============================] - 0s 2ms/step - loss: 0.1573 - accuracy: 0.9894
[0.23694023089626984, 0.4662767603313257, 0.4655859531486418, 0.16025283179814637, 0.35186587230994304, 0.19934259440768637, 0.20519602623672575, 0.38171548184674114, 0.5406411164158526, 0.20698360034189725, 0.1318

middle of dataset

In [ ]:
train = True
modelList = ['baseline','neuralFS','pairwise','cancelOut','regularized']
result = {}
result["shape"] = X_train.shape
if train:
    for key in modelList:
        if key != "filter":
            ind = crossStab(key,4,X_train,y_train,mapModel, params, percent=False, nFeat=int((X_train.shape[1])/2), epc=100, bs=32, verb=0)
            result["stab_"+key] = ind
writeOut("resultFS.txt",params,result,dfstr)

6/6 [==============================] - 0s 2ms/step - loss: 0.1884 - accuracy: 0.9681
[0.5663455900013835, 0.24552549190855188, 0.2529575803299541, 0.18797276976252203, 0.4118627686958797, 0.1696782581637814, 0.30028355039146815, 0.256406654968838, 0.3152196550750397, 0.2557079687165441, 0.18917003647458697, 0.18388536130609417, 0.136729119503003, 0.0793985047991486, 0.18082031643260377, 0.1814674019292795, 0.05993404999758082, 0.25502449867322796, 0.14152177876519695, 0.08535228020121961, 0.52537642248828, 0.619632958490834, 0.23374796758610045, 0.2889678833966555, 0.8822056667239496, 0.224882363319099, 0.26426613398815707, 0.41791504684525627, 0.2968990106250692, 0.1529778009772028]
6/6 [==============================] - 0s 2ms/step - loss: 0.1593 - accuracy: 0.9840
[0.3718059798472479, 0.30016022846779783, 0.22828652114618894, 0.1854487211696249, 0.38084647884259654, 0.25176646259713864, 0.3003487165106339, 0.41945255082091026, 0.33534601388751967, 0.2615572053665105, 0.1625704900224

In [ ]:
def result_stamp(stamp, params):
  #print(params)
  print(params.keys())
  new_params = {}
  for key in params.keys():
    print("heyos",key)
    new_key = key+stamp
    new_params[new_key] = params[key]
  return new_params

In [ ]:
def testAll(modelList,params, stamp):
  scores=[]
  print(modelList)
  print("100% OF FEATURES")
  result = testModels(X_train, y_train, modelList, params, epc=100, percent=True, nFeat=100, showFeat=False, cols=False)
  pprint.pprint(params)
  print("___________________________________")
  result = result_stamp(stamp,result)
  pprint.pprint(result)
  writeOut("resultFS.txt",params,result,dfstr)
  print("#######################################################")
  print("75% OF FEATURES")
  result = testModels(X_train, y_train, modelList, params, epc=100, percent=True, nFeat=75, showFeat=False, cols=False)
  pprint.pprint(params)
  print("___________________________________")
  result = result_stamp(stamp,result)
  pprint.pprint(result)
  writeOut("resultFS.txt",params,result,dfstr)
  print("#######################################################")
  print("Stability 100%")
  result = {}
  result["shape"] = X_train.shape
  for key in modelList:
    ind = crossStab(key,4,X_train,y_train,mapModel, params, percent=True, nFeat=100, epc=100, bs=32, verb=0)
    result["stab_"+key+"_"+stamp] = ind
  writeOut("resultFS.txt",params,result,dfstr)
  print("#######################################################")
  result = {}
  result["shape"] = X_train.shape
  print("Stability Mid")
  for key in modelList:
    ind = crossStab(key,4,X_train,y_train,mapModel, params, percent=False, nFeat=int((X_train.shape[1])/2), epc=100, bs=32, verb=0)
    result["stab_"+key+"_"+stamp] = ind
  writeOut("resultFS.txt",params,result,dfstr)
  print("#######################################################")

Influence of L1 param


In [ ]:
paramz = params.copy()
modelList = ["cancelOut","pairwise","regularized"]
for i in [0.01,0.1,0.3,0.5,0.8]:
  paramz["l1_cancelOut"] = i
  paramz["l1_pairwise"] = i
  paramz["l1_regul"] = i
  testAll(modelList,paramz,"l1")

['cancelOut', 'pairwise', 'regularized']
100% OF FEATURES
6/6 [==============================] - 0s 2ms/step - loss: 0.1479 - accuracy: 0.9787
$$__cancelOut
6/6 [==============================] - 0s 2ms/step - loss: 0.1941 - accuracy: 0.9681
[2.2418886e-05, 1.4480897e-05, 9.764301e-06, 3.3809414e-05, 7.3533406e-06, 9.142777e-05, 2.3688892e-05, 5.6919616e-05, 1.22397505e-05, 2.062875e-05, 1.8484903e-05, 3.420679e-05, 1.911991e-05, 8.3151e-05, 1.4275483e-05, 5.2187993e-06, 2.061903e-05, 1.643246e-05, 3.8133174e-05, 1.34598495e-05, 3.4989139e-06, 0.0001082813, 7.764386e-05, 1.2132109e-05, 1.0500871e-05, 2.33972e-05, 4.8039565e-06, 5.064301e-05, 3.1074822e-05, 9.661207e-06]
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, False, True, True, True]


$$__pairwise
6/6 [==============================] - 0s 2ms/step - loss: 0.2946 - accuracy: 0.9681
[0.012577379, 0.053625897, 0.001518097

Influence of L2 param


In [ ]:
paramz = params.copy()
modelList = ["cancelOut","pairwise", "neuralFS"]
for i in [0.01,0.1,0.3,0.5,0.8]:
  paramz["l2_cancelOut"] = i
  paramz["l2_pairwise"] = i
  paramz["l2_neuralfs"] = i
  testAll(modelList,paramz, "l2")

['cancelOut', 'pairwise', 'neuralFS']
100% OF FEATURES
6/6 [==============================] - 0s 2ms/step - loss: 0.1825 - accuracy: 0.9628
$$__cancelOut
6/6 [==============================] - 0s 2ms/step - loss: 0.1878 - accuracy: 0.9681
[4.336139e-05, 1.0526623e-05, 7.4177966e-05, 0.00012306128, 8.739371e-05, 4.151084e-05, 5.9409966e-05, 0.00022212401, 6.545557e-05, 0.00013833126, 0.00014118501, 0.00022258516, 7.1127426e-05, 0.0002086837, 0.0003078351, 5.224224e-05, 8.457983e-05, 0.00011701245, 0.00013945586, 0.00012306332, 0.00011326402, 0.0002990379, 0.00017187561, 1.8306378e-05, 0.0001459833, 0.0003165667, 0.00015764474, 9.3663235e-05, 0.00024928834, 0.00012336278]
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


$$__pairwise
6/6 [==============================] - 0s 2ms/step - loss: 0.2623 - accuracy: 0.9521
[0.00016536484, 8.87517e-05, 0.000218200

Influence of L3 param


In [ ]:
paramz = params.copy()
modelList =["cancelOut","pairwise","regularized"]
for i in [0.01,0.1,0.3,0.5,0.8]:
  paramz["l3_cancelOut"] = i 
  paramz["l3_pairwise"] = i
  testAll(modelList,paramz, "l3")

['cancelOut', 'pairwise', 'regularized']
100% OF FEATURES
6/6 [==============================] - 0s 2ms/step - loss: 0.1675 - accuracy: 0.9787
$$__cancelOut
6/6 [==============================] - 0s 2ms/step - loss: 0.1928 - accuracy: 0.9574
[0.00034268157, 1.2762662e-05, 0.00036529687, 7.296135e-05, 2.086014e-06, 0.00035049237, 0.00036370842, 8.901385e-05, 0.00016833295, 1.5072725e-05, 0.00034058158, 6.0215887e-05, 0.00033248067, 0.00016640664, 4.2533124e-05, 3.374954e-05, 5.775326e-05, 8.497229e-05, 5.9245092e-05, 6.934561e-05, 0.00030329303, 0.00017855821, 0.00017423222, 0.0003469137, 0.00021263139, 7.1669696e-05, 0.00033722445, 0.00032113647, 0.0003606549, 2.9450242e-05]
[True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


$$__pairwise
6/6 [==============================] - 0s 2ms/step - loss: 0.2924 - accuracy: 0.9734
[0.0006050713, 0.00891428, 0.00013

Influence of threshold


In [ ]:
paramz = params.copy()
modelList = ["neuralFS"]
for i in [0.01,0.1,0.3,0.5,0.8]:
  paramz["threshold_neuralfs"] = i 
  testAll(modelList,paramz, "threshold")

['neuralFS']
100% OF FEATURES
6/6 [==============================] - 0s 2ms/step - loss: 0.2445 - accuracy: 0.9468
$$__neuralFS
6/6 [==============================] - 0s 2ms/step - loss: 0.1729 - accuracy: 0.9681
[0.12936103, 0.26335725, 0.123971745, 0.17085081, 0.0, 0.019215088, 0.11734433, 0.27568397, 0.06613537, 0.093214855, 0.47911468, 0.0, 0.3449009, 0.551775, 0.021124415, 0.011117271, 0.039378032, 0.13656865, 0.17131825, 0.37158853, 0.23344237, 0.2945499, 0.22888502, 0.3504584, 0.24122147, 0.08686394, 0.22008537, 0.24597722, 0.17458348, 0.11310228]
[True, True, True, True, False, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


{'dense1_baseline': 10,
 'dense2_baseline': 50,
 'dense3_baseline': 100,
 'initialiser_cancelOut': 'const',
 'initialiser_pairwise': 'const',
 'k': 3,
 'l1_baseline': 0.001,
 'l1_cancelOut': 0.5,
 'l1_neuralfs': 0.0,
 'l1_pairwise': 0.05,
 'l1_regul': 0.